# PSP-Routing-System: Intelligente Zahlungsdienstleister-Empfehlung

## Methodischer Ansatz

### Datengrundlage
- Transaktionsdaten aus PSP_Jan_Feb_2019.xlsx
- Historische Erfolgs- und Kostendaten von vier PSPs: Goldcard, Moneycard, Simplecard, UK_Card

### Modellarchitektur
**Klassifikationsmodell (XGBoost):**
- Vorhersage der Erfolgswahrscheinlichkeit je PSP
- Gewichtung: 70% im Gesamtscore
- PSP-spezifische Thresholds als Qualitätsfilter

**Regressionsmodell (LightGBM):**
- Schätzung der Transaktionskosten je PSP (Skala 0-10)
- Gewichtung: 30% im Gesamtscore (invertiert)

### Feature Engineering
- Zeitbasierte Features: Stunde, Tageszeit, Monatsanfang
- Transaktionsmerkmale: Betrag, Kartentyp, 3D-Secured-Status
- Zyklische Kodierung und Kategorisierung

### Deployment
Bereitstellung über Gradio-Interface für Echtzeit-Vorhersagen mit wahlweise detaillierter oder kompakter Ausgabe.

## Notebook-Struktur
1. Imports und Datenaufbereitung
2. Modell mit detaillierter Ausgabe (alle Scores und Metriken)
3. Modell mit kompakter Ausgabe (nur PSP-Name)
4. Dokumentation der Pipeline-Ablauf


# Imports

In [1]:
import gradio as gr
import datetime
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler

# Modellanwendung


## Modell mit Infos in der Ausgabe

Dieses Interface zeigt alle berechneten Zwischenschritte und Metriken:
- Klassifikationswahrscheinlichkeiten aller PSPs
- Threshold-Entscheidungen (qualifiziert/nicht qualifiziert)
- Regressionscores (Kosten)
- Finale Gesamtscores
- Fallback-Status

Geeignet für: Analyse, Debugging, Modellvalidierung

In [2]:
import gradio as gr
import datetime
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler


raw_path = Path(r"../Data/PSP_Jan_Feb_2019.xlsx")
df = pd.read_excel(raw_path)

# === MODELLE LADEN ===
xgb_model = pickle.load(open("XGBoost_auc_enchanting-bee-331.pkl", 'rb'))
lgbm_model_10 = pickle.load(open("lgbm10_crawling-boar-12_d1ad86285a0b4868868f300261aa4364.pkl", 'rb'))

# Scaler außerhalb fitten (einmalig)
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(df[['amount']])

# Amount Bins berechnen (einmalig)
amount_bins = pd.qcut(df['amount'], q=4, labels=[1,2,3,4], retbins=True, duplicates='drop')[1]

# PSP Thresholds definieren (Entscheidungsgrenzen)
PSP_THRESHOLDS = {
    "Simplecard": 0.51,
    "Moneycard": 0.55,
    "Goldcard": 0.60,
    "UK_Card": 0.47
}


def predict_psp_scores(amount, secured, card):
    """
    Hauptfunktion: Berechnet Gesamtscore aus Klassifikation (70%) und Regression (30%).
    Verwendet PSP-spezifische Thresholds als Qualitätsfilter mit Fallback.
    """
    # 1. Feature Engineering
    tmsp = datetime.datetime.now()
    hour = tmsp.hour
    monatsanfang = 1 if tmsp.day <= 3 or tmsp.day >= 29 else 0
    
    daytime_nachmittag = 1 if 13 <= hour < 19 else 0
    daytime_nacht = 1 if 0 <= hour < 7 else 0
    hour_sin = np.sin(2 * np.pi * hour/24.0)
    
    oh__card_Diners = 1 if card == 'Diners' else 0
    oh__card_Visa = 1 if card == 'Visa' else 0
    oh__card_Master = 1 if card == 'Master' else 0
    
    amount_cat = pd.cut([amount], bins=amount_bins, labels=[1,2,3,4])[0]
    bin__3D_secured = 1 if secured == 'Ja' else 0
    remainder__amount_scaled = scaler.transform(pd.DataFrame([[amount]], columns=['amount']))[0][0]
    
    # === REGRESSION SCORES ===
    input_dict_regression = {
        'bin__3D_secured': bin__3D_secured,
        'bin__daytime_Nachmittag': daytime_nachmittag,
        'bin__daytime_Nacht': daytime_nacht,
        'monatsanfang': monatsanfang,
        'oh__PSP_Goldcard': 0,
        'oh__PSP_Moneycard': 0,
        'oh__PSP_Simplecard': 0,
        'oh__PSP_UK_Card': 0,
        'oh__card_Diners': oh__card_Diners,
        'oh__card_Visa': oh__card_Visa,
        'ord__amount_cat': amount_cat,
        'remainder__amount_scaled': remainder__amount_scaled,
        'remainder__hour_sin': hour_sin
    }
    
    psp_names = ['Goldcard', 'Moneycard', 'Simplecard', 'UK_Card']
    psp_columns = [f'oh__PSP_{name}' for name in psp_names]
    
    X_base_reg = pd.DataFrame([input_dict_regression])
    regression_scores = {}
    
    for psp_name, psp_col in zip(psp_names, psp_columns):
        X_hypo = X_base_reg.copy()
        
        for col in psp_columns:
            X_hypo[col] = 0
        
        X_hypo[psp_col] = 1
        pred_value = lgbm_model_10.predict(X_hypo)[0]
        
        # Skalierung von [0, 10] auf [0, 1]
        regression_scores[psp_name] = float(np.clip(pred_value / 10, 0, 1))
    
    # === KLASSIFIKATION WAHRSCHEINLICHKEITEN ===
    input_dict_classification = {
        'oh__card_Visa': oh__card_Visa,
        'oh__PSP_Simplecard': 0,
        'oh__card_Master': oh__card_Master,
        'oh__card_Diners': oh__card_Diners,
        'monatsanfang': monatsanfang,
        'bin__daytime_Nachmittag': daytime_nachmittag,
        'bin__daytime_Nacht': daytime_nacht,
        'ord__amount_cat': amount_cat,
        'bin__3D_secured': bin__3D_secured,
        'oh__PSP_Moneycard': 0,
        'oh__PSP_Goldcard': 0,
        'remainder__hour_sin': hour_sin,
        'remainder__amount_scaled': remainder__amount_scaled,
        'oh__PSP_UK_Card': 0
    }
    
    X_base_clf = pd.DataFrame([input_dict_classification])
    classification_probs = {}
    classification_decisions = {}
    
    for psp_name in psp_names:
        X_hypo = X_base_clf.copy()
        
        # Alle PSPs auf 0 setzen
        for psp in psp_names:
            X_hypo[f'oh__PSP_{psp}'] = 0
        
        # Aktuelle PSP auf 1 setzen
        X_hypo[f'oh__PSP_{psp_name}'] = 1
        
        # Vorhersage der Wahrscheinlichkeit
        prob = float(xgb_model.predict_proba(X_hypo)[:, 1][0])
        classification_probs[psp_name] = prob
        
        # Threshold-Check
        classification_decisions[psp_name] = 1 if prob >= PSP_THRESHOLDS[psp_name] else 0
    
    # === GESAMTSCORE BERECHNEN ===
    # Prüfe, ob mindestens eine PSP den Threshold überschreitet
    any_qualified = any(classification_decisions.values())
    
    final_scores = {}
    for psp_name in psp_names:
        if any_qualified:
            # Normalfall: Nur qualifizierte PSPs bekommen Score
            if classification_decisions[psp_name] == 1:
                final_scores[psp_name] = (
                    classification_probs[psp_name] * 0.7 - 
                    regression_scores[psp_name] * 0.3
                )
            else:
                final_scores[psp_name] = None
        else:
            # Fallback: Keine PSP qualifiziert → Berechne für ALLE
            final_scores[psp_name] = (
                classification_probs[psp_name] * 0.7 - 
                regression_scores[psp_name] * 0.3
            )
    
    # === BESTE PSP ERMITTELN ===
    valid_scores = {psp: score for psp, score in final_scores.items() if score is not None}
    
    if valid_scores:
        best_psp = max(valid_scores, key=valid_scores.get)
        best_score = valid_scores[best_psp]
        fallback_used = not any_qualified
    else:
        # Sollte nie passieren
        best_psp = "Fehler"
        best_score = None
        fallback_used = False
    
    # === ERGEBNIS MIT ALLEN DETAILS ===
    result = {
        "Klassifikation_Wahrscheinlichkeiten": classification_probs,
        "Klassifikation_Entscheidungen (0=abgelehnt, 1=qualifiziert)": classification_decisions,
        "Regression_Scores": regression_scores,
        "Gesamtscores": final_scores,
        "Empfohlene_PSP": best_psp,
        "Bester_Score": round(best_score, 4) if best_score is not None else "Fehler",
        "Fallback_Modus": fallback_used,
        "Info": "Fallback aktiv: Keine PSP erreicht Threshold" if fallback_used else "Normale Empfehlung: Mindestens eine PSP qualifiziert",
        "Angewendete_Thresholds": PSP_THRESHOLDS
    }
    
    return result


# === GRADIO INTERFACE ===
with gr.Blocks() as demo:
    gr.Markdown("# PSP Gesamtscore Vorhersage")
    gr.Markdown("**Gesamtscore = Klassifikation (70%) - Regression (30%)**")
    gr.Markdown("*PSPs werden bevorzugt empfohlen, wenn ihre Erfolgswahrscheinlichkeit den Threshold überschreitet*")
    gr.Markdown("**Fallback: Wenn keine PSP qualifiziert ist, wird die beste verfügbare PSP empfohlen**")
    gr.Markdown("**Thresholds: Simplecard=0.51 | Moneycard=0.55 | Goldcard=0.60 | UK_Card=0.47**")
    
    with gr.Row():
        amount_input = gr.Number(
            label="Betrag (Amount)",
            value=0.0,
            precision=2
        )
    
    with gr.Row():
        secured_input = gr.Radio(
            choices=["Ja", "Nein"],
            label="3D Secured",
            value="Nein"
        )
    
    with gr.Row():
        card_input = gr.Dropdown(
            choices=["Visa", "Diners", "Master"],
            label="Kartentyp (Card)",
            value="Visa"
        )
    
    submit_btn = gr.Button("Vorhersage starten")
    output = gr.JSON(label="PSP Gesamtscores")
    
    submit_btn.click(
        fn=predict_psp_scores,
        inputs=[amount_input, secured_input, card_input],
        outputs=output
    )

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Modell mit einer Ausgabe: Bester PSP

Minimalistisches Interface für Produktivumgebungen:
- Gibt nur den Namen der empfohlenen PSP zurück
- Kompakte Benutzeroberfläche
- Geeignet für API-Integration und Live-Systeme

In [3]:
import gradio as gr
import datetime
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler


raw_path = Path(r"../Data/PSP_Jan_Feb_2019.xlsx")
df = pd.read_excel(raw_path)

# === MODELLE LADEN ===
xgb_model = pickle.load(open("XGBoost_auc_enchanting-bee-331.pkl", 'rb'))
lgbm_model_10 = pickle.load(open("lgbm10_crawling-boar-12_d1ad86285a0b4868868f300261aa4364.pkl", 'rb'))

# Scaler außerhalb fitten (einmalig)
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(df[['amount']])

# Amount Bins berechnen (einmalig)
amount_bins = pd.qcut(df['amount'], q=4, labels=[1,2,3,4], retbins=True, duplicates='drop')[1]

# PSP Thresholds definieren (Entscheidungsgrenzen)
PSP_THRESHOLDS = {
    "Simplecard": 0.51,
    "Moneycard": 0.55,
    "Goldcard": 0.60,
    "UK_Card": 0.47
}


def predict_psp_scores(amount, secured, card):
    """
    Hauptfunktion: Berechnet Gesamtscore aus Klassifikation (70%) und Regression (30%).
    Verwendet PSP-spezifische Thresholds als Qualitätsfilter mit Fallback.
    """
    # 1. Feature Engineering
    tmsp = datetime.datetime.now()
    hour = tmsp.hour
    monatsanfang = 1 if tmsp.day <= 3 or tmsp.day >= 29 else 0
    
    daytime_nachmittag = 1 if 13 <= hour < 19 else 0
    daytime_nacht = 1 if 0 <= hour < 7 else 0
    hour_sin = np.sin(2 * np.pi * hour/24.0)
    
    oh__card_Diners = 1 if card == 'Diners' else 0
    oh__card_Visa = 1 if card == 'Visa' else 0
    oh__card_Master = 1 if card == 'Master' else 0
    
    amount_cat = pd.cut([amount], bins=amount_bins, labels=[1,2,3,4])[0]
    bin__3D_secured = 1 if secured == 'Ja' else 0
    remainder__amount_scaled = scaler.transform(pd.DataFrame([[amount]], columns=['amount']))[0][0]
    
    # === REGRESSION SCORES ===
    input_dict_regression = {
        'bin__3D_secured': bin__3D_secured,
        'bin__daytime_Nachmittag': daytime_nachmittag,
        'bin__daytime_Nacht': daytime_nacht,
        'monatsanfang': monatsanfang,
        'oh__PSP_Goldcard': 0,
        'oh__PSP_Moneycard': 0,
        'oh__PSP_Simplecard': 0,
        'oh__PSP_UK_Card': 0,
        'oh__card_Diners': oh__card_Diners,
        'oh__card_Visa': oh__card_Visa,
        'ord__amount_cat': amount_cat,
        'remainder__amount_scaled': remainder__amount_scaled,
        'remainder__hour_sin': hour_sin
    }
    
    psp_names = ['Goldcard', 'Moneycard', 'Simplecard', 'UK_Card']
    psp_columns = [f'oh__PSP_{name}' for name in psp_names]
    
    X_base_reg = pd.DataFrame([input_dict_regression])
    regression_scores = {}
    
    for psp_name, psp_col in zip(psp_names, psp_columns):
        X_hypo = X_base_reg.copy()
        
        for col in psp_columns:
            X_hypo[col] = 0
        
        X_hypo[psp_col] = 1
        pred_value = lgbm_model_10.predict(X_hypo)[0]
        
        # Skalierung von [0, 10] auf [0, 1]
        regression_scores[psp_name] = float(np.clip(pred_value / 10, 0, 1))
    
    # === KLASSIFIKATION WAHRSCHEINLICHKEITEN ===
    input_dict_classification = {
        'oh__card_Visa': oh__card_Visa,
        'oh__PSP_Simplecard': 0,
        'oh__card_Master': oh__card_Master,
        'oh__card_Diners': oh__card_Diners,
        'monatsanfang': monatsanfang,
        'bin__daytime_Nachmittag': daytime_nachmittag,
        'bin__daytime_Nacht': daytime_nacht,
        'ord__amount_cat': amount_cat,
        'bin__3D_secured': bin__3D_secured,
        'oh__PSP_Moneycard': 0,
        'oh__PSP_Goldcard': 0,
        'remainder__hour_sin': hour_sin,
        'remainder__amount_scaled': remainder__amount_scaled,
        'oh__PSP_UK_Card': 0
    }
    
    X_base_clf = pd.DataFrame([input_dict_classification])
    classification_probs = {}
    classification_decisions = {}
    
    for psp_name in psp_names:
        X_hypo = X_base_clf.copy()
        
        # Alle PSPs auf 0 setzen
        for psp in psp_names:
            X_hypo[f'oh__PSP_{psp}'] = 0
        
        # Aktuelle PSP auf 1 setzen
        X_hypo[f'oh__PSP_{psp_name}'] = 1
        
        # Vorhersage der Wahrscheinlichkeit
        prob = float(xgb_model.predict_proba(X_hypo)[:, 1][0])
        classification_probs[psp_name] = prob
        
        # Threshold-Check
        classification_decisions[psp_name] = 1 if prob >= PSP_THRESHOLDS[psp_name] else 0
    
    # === GESAMTSCORE BERECHNEN ===
    # Prüfe, ob mindestens eine PSP den Threshold überschreitet
    any_qualified = any(classification_decisions.values())
    
    final_scores = {}
    for psp_name in psp_names:
        if any_qualified:
            # Normalfall: Nur qualifizierte PSPs bekommen Score
            if classification_decisions[psp_name] == 1:
                final_scores[psp_name] = (
                    classification_probs[psp_name] * 0.7 - 
                    regression_scores[psp_name] * 0.3
                )
            else:
                final_scores[psp_name] = None
        else:
            # Fallback: Keine PSP qualifiziert → Berechne für ALLE
            final_scores[psp_name] = (
                classification_probs[psp_name] * 0.7 - 
                regression_scores[psp_name] * 0.3
            )
    
    # === BESTE PSP ERMITTELN ===
    valid_scores = {psp: score for psp, score in final_scores.items() if score is not None}
    
    if valid_scores:
        best_psp = max(valid_scores, key=valid_scores.get)
        best_score = valid_scores[best_psp]
        fallback_used = not any_qualified
    else:
        # Sollte nie passieren
        best_psp = "Fehler"
        best_score = None
        fallback_used = False
    
    # === NUR BESTE PSP ZURÜCKGEBEN ===
    return best_psp


# === GRADIO INTERFACE ===
with gr.Blocks() as demo:
    gr.Markdown("# PSP Empfehlung")
    
    with gr.Row():
        amount_input = gr.Number(
            label="Betrag (Amount)",
            value=0.0,
            precision=2
        )
    
    with gr.Row():
        secured_input = gr.Radio(
            choices=["Ja", "Nein"],
            label="3D Secured",
            value="Nein"
        )
    
    with gr.Row():
        card_input = gr.Dropdown(
            choices=["Visa", "Diners", "Master"],
            label="Kartentyp (Card)",
            value="Visa"
        )
    
    submit_btn = gr.Button("PSP empfehlen")
    output = gr.Textbox(label="Empfohlene PSP", interactive=False)
    
    submit_btn.click(
        fn=predict_psp_scores,
        inputs=[amount_input, secured_input, card_input],
        outputs=output
    )

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## Ablauf der PSP-Vorhersage-Pipeline

### 1. Dateninitialisierung
- Laden des Excel-Datensatzes aus `PSP_Jan_Feb_2019.xlsx`
- Laden zweier trainierter Modelle: XGBoost für Klassifikation und LightGBM für Regression
- Initialisierung eines MinMaxScalers auf Basis der Amount-Spalte des Datensatzes
- Berechnung der Quantil-Grenzen für die Kategorisierung von Beträgen in 4 Klassen
- Definition PSP-spezifischer Thresholds (Simplecard: 0.51, Moneycard: 0.55, Goldcard: 0.60, UK_Card: 0.47)

### 2. Feature Engineering
Bei jedem Funktionsaufruf werden folgende Features aus den Eingabedaten generiert:

**Zeitbasierte Features:**
- Aktuelle Stunde für zyklische Kodierung
- Monatsanfang-Indikator bei Tagen 1-3 oder 29-31
- Tageszeit-Kategorien für Nachmittag (13-19 Uhr) und Nacht (0-7 Uhr)
- Sinusförmige Transformation der Stunde für zyklische Zeitrepräsentation

**Kartentyp-Features:**
- One-Hot-Kodierung für Visa, Diners und Master

**Betragsbezogene Features:**
- Kategorisierung des Betrags in 4 Quantilgruppen
- MinMax-Skalierung des Betrags auf Wertebereich 0-1

**Sicherheitsfeature:**
- Binäre Kodierung der 3D-Secured-Option

### 3. Regressionsmodell-Pipeline
- Erstellung eines Feature-Dictionaries mit 13 Features
- Iteration über alle 4 PSPs (Goldcard, Moneycard, Simplecard, UK_Card)
- Für jede PSP: Setzen aller PSP-Features auf 0, nur die aktuelle PSP auf 1
- Vorhersage des Regressionswertes (0-10 Skala, repräsentiert Transaktionskosten)
- Normalisierung auf 0-1 durch Division mit 10 und Clipping

### 4. Klassifikationsmodell-Pipeline
- Erstellung eines separaten Feature-Dictionaries mit 14 Features
- Iteration über alle 4 PSPs
- Für jede PSP: Manipulation der PSP-Features analog zur Regression
- Berechnung der Erfolgswahrscheinlichkeit mittels predict_proba
- Extraktion der Wahrscheinlichkeit für die positive Klasse
- Anwendung PSP-spezifischer Thresholds als Qualitätsfilter

#### 4.1 Threshold-basierte Filterung
- Vergleich jeder Erfolgswahrscheinlichkeit mit dem PSP-spezifischen Threshold
- Qualifizierung nur der PSPs, die ihren Threshold erreichen oder überschreiten
- Fallback-Mechanismus: Falls keine PSP qualifiziert ist, werden alle PSPs bewertet

### 5. Gesamtscore-Berechnung
- Kombination beider Modelle nach der Formel: Klassifikation × 0.7 - Regression × 0.3
- Die Subtraktion reflektiert, dass niedrigere Regressionswerte (Kosten) besser sind
- Berechnung nur für qualifizierte PSPs (außer im Fallback-Modus)
- Ermittlung der PSP mit dem höchsten Gesamtscore

### 6. Output-Strukturierung
Rückgabe des PSP-Namens als String:
- Name der empfohlenen PSP (z.B. "Simplecard")
- Minimalistisches Output-Format für direkte Integration in Produktivsysteme

### 7. Gradio-Interface
- Drei Eingabefelder: Betrag (Number), 3D-Secured (Radio), Kartentyp (Dropdown)
- Button "PSP empfehlen" zum Starten der Vorhersage
- Textbox-Output zur Anzeige der empfohlenen PSP
- Kompakte Benutzeroberfläche für schnelle Entscheidungen